In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skvideo.io
import skvideo.datasets
import os
from copy import deepcopy as copy

In [38]:
class chaotic_pixels:
    def __init__(self):
        self.x_1 = 0.2350
        self.y_1 = 0.3500
        self.z_1 = 0.7350
        self.v_1 = (self.x_1, self.y_1, self.z_1)
        
        self.alpha = 0.0125 # 0 < alpha < 0.015
        self.beta = 0.0157  # 0 < beta < 0.022
        self.gamma = 3.7700 # 3.53 < gamma < 3.81
        
        self.N = 10**15
    
    def logistic_chaos_map(self, point):
        xn, yn, zn = point
        x = (self.gamma*xn*(1-xn)) + (self.beta*(yn**2)*xn) + (self.alpha*(zn**3))
        y = (self.gamma*yn*(1-yn)) + (self.beta*(zn**2)*yn) + (self.alpha*(xn**3))    
        z = (self.gamma*zn*(1-zn)) + (self.beta*(xn**2)*zn) + (self.alpha*(yn**3))
    
        return x, y, z
    
    def normalize(self, point, video_dims):
        x, y, z = point
        return (np.int64(x*self.N)%video_dims[0], np.int64(y*self.N)%video_dims[1], np.int64(z*self.N)%video_dims[2])
    
    def get_pixels(self, video_dims, data_len, get_reps=False):
        prev = self.v_1
        values = []
        hash_table = {}
        count = 0
        reps = []
        for i in range(0, data_len):
            next_point = self.logistic_chaos_map(prev)
            n_next = self.normalize(next_point, video_dims)
            prev = next_point
            
            key = hash(n_next)
            if key in hash_table:
                if n_next in hash_table[key]:
                    reps.append(i)
                    if(len(reps)%100 == 0):
                        print(reps[-1])
                    continue
                else:
                    hash_table[key].append(n_next)
            else:
                hash_table[key] = [n_next]
        
            values.append(n_next)
            
        values = np.array(values, dtype=np.int64)
        ids = np.arange(0, data_len)
        values = np.c_[values, ids]
        values = values[values[:,0].argsort()]
        
        if get_reps:
            return values, reps
        return values

In [39]:
gen = chaotic_pixels()
gen.get_pixels((100,100,100), 10)

array([[ 0,  0,  0,  0],
       [ 0, 57, 57,  8],
       [ 9, 58, 89,  2],
       [23,  8, 61,  4],
       [24, 70, 89,  5],
       [68, 34, 66,  1],
       [80, 76, 99,  6],
       [84, 78, 15,  9],
       [89, 84, 19,  3],
       [89, 84, 91,  7]])

In [37]:
(256*256*256)/(1024*1024)

16.0